In [1]:
from ipywidgets import *
from ipywebrtc import CameraStream, ImageRecorder
from IPython.display import IFrame, YouTubeVideo
from IPython.core.display import display
from ipywidgets import HTML
import subprocess, sys
import base64
import cv2
import io
import numpy as np
import PIL.Image
from PIL import *

In [2]:
gamma = 1
contrast_init = 0

In [3]:
####  Inversion  #####
def invert(_):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    #########################################
    iminvers = cv2.bitwise_not(imm)
    ####################################
    fmt = 'png'
    data_comp = compress_to_bytes(iminvers, fmt)
    Img.value=data_comp

In [4]:
#################################
def Canny_transfo(_):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    #########################################"

    # Setting parameter values
    t_lower = 50  # Lower Threshold
    t_upper = 150  # Upper threshold

    # Applying the Canny Edge filter
    edge = cv2.Canny(imm, t_lower, t_upper)
    ####################################
    fmt = 'png'
    data_comp = compress_to_bytes(edge, fmt)
    Img.value=data_comp

In [5]:
#################################
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########

In [6]:
#############################################
def adjust_brightness(img, brightness_factor):
    print("brightness_factor = ",brightness_factor)
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################

In [7]:
def Travail_img(gamma, contrast_init):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    imm=adjust_brightness(imm, gamma)
    #########################################"
    fmt = 'png'
    data_comp = compress_to_bytes(imm, fmt)
    Img.value=data_comp
    return gamma, contrast_init
    #####

In [8]:
def Contrast_w(_):
    global gamma, contrast_init
    contrast=contraste_widg.value
#     print("contraste lu = ",contrast)
#     print("contrast_init = ",contrast_init)
    if contrast > contrast_init:

        gamma=float((contrast-contrast_init)/10)+1
#         print("* Gamma = ",gamma)
        contrast_init = contrast
#         print("************** New contrast-init = ",contrast_init)
    else:
        gamma=1-float((contrast_init-contrast)/10)
#         print("* Gamma2 = ",gamma)
        contrast_init = contrast
#         print("************** New contrast-init2 = ",contrast_init)
    
    Travail_img(gamma, contrast_init)

In [9]:
######  Recup Image d'1 Vidéo  ########
def Recup_image(_):
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)#[..., :3]
    ####
#     mask_circle = np.zeros(im_array.shape[:2],np.uint8)
#     cv2.circle(mask_circle,(320,240), 50, 255, -1) # a.value, 255, -1)
#     img_rgb = cv2.bitwise_and(im_array, im_array, mask=mask_circle)    
    ####
    fmt = 'png'
    data_comp = compress_to_bytes(im_array, fmt)
    Img.value = data_comp
    Img_rescue.value = data_comp

    ####
#     data_comp2 = compress_to_bytes(im_array, fmt)
#     image_headline2.value = data_comp2
    yo.layout.visibility = 'hidden'
#     Mode_w.value="---"
#################################

In [10]:
#######################################
def upload_func(change):
    valeur=change['new']
#     global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode, Trait,Down
    if valeur=={}:
        rien=1
    else:
        im_val = uplo.value
        cles = list(im_val.keys())
#         print(cles)
        cle_one=cles[0]
        conta = im_val[cle_one]
        contenu_ex_taille = conta['content']
        nparr = np.frombuffer(contenu_ex_taille, np.uint8)
        img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)  # cv2.CV_LOAD_IMAGE_COLOR) # cv2.IMREAD_COLOR in OpenCV 3.1   
        pil_im=cv2.resize(img_np, (640,480), interpolation = cv2.INTER_NEAREST)
        pil_im = cv2.cvtColor(pil_im, cv2.COLOR_BGR2RGB)
#         shap = pil_im.shape

        pil_array=np.array(pil_im)
        fmt = 'png'
        data_image_headline = compress_to_bytes(pil_array, fmt)
            
        Img.value=data_image_headline
        Img_rescue.value=data_image_headline 

#         Iris_ok=1
        Travail_img(gamma, contrast_init)
#############################################

In [11]:
##########
def arrete_tout(_):
    Widget.close_all()

In [12]:
###########
def reinitialise(_):
    Img.value = Img_rescue.value

In [13]:
###########
def Capture_w_fonc(_):
    yo.layout.display = "block"

<div style="text-align:center;  font-size: 3.2em; width:70%;">
<span style="color:brown"><b><u>APPLICATIONS EN LIGNE</u></b></span>
</div>



<div style="text-align:center; border:2px solid black; font-size: 1.8em; width:40%;">
<span style="color:blue"><b>Application Web de retouche d'images :</b></span>
</div>



In [14]:
file = open("Lycee_NSI.jpg", "rb")
image = file.read()
Img = widgets.Image(
    value=image,
    format='jpg',
    width=600,
    height=550,
)
Img_rescue = widgets.Image(
    value=image,
    format='jpg',
    width=600,
    height=550,
)

In [15]:
contraste_widg = widgets.IntSlider(min = -10, max = +10,  value = 0 , description='+ clair')
contraste_widg.observe(Contrast_w, names='value')

In [16]:
#### Stop  ####
Stop_w = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour fermer l'Appli",
    icon='times-circle' # (FontAwesome names without the `fa-` prefix)
)
Stop_w.observe(arrete_tout,names='value')
#### Rescue  ####
Rescue_w = widgets.ToggleButton(
    value=False,
    description='Réinitialise',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour revenir à l'image référence",
    icon='undo' # (FontAwesome names without the `fa-` prefix)
)
Rescue_w.observe(reinitialise,names='value')
#####################

In [17]:
####  Dessin  #####
Cany_w = widgets.ToggleButton(
    value=False,
    description='Dessin',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour obtenir le dessin de contour",
    icon='pencil-alt' # (FontAwesome names without the `fa-` prefix)
)
Cany_w.observe(Canny_transfo,names='value')
#####################

In [18]:
####  Invert  #####
Invert_w = widgets.ToggleButton(
    value=False,
    description='Inverse',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour inverser le noir et le blanc",
    icon='pencil-alt' # (FontAwesome names without the `fa-` prefix)
)
Invert_w.observe(invert,names='value')
#####################

In [19]:
###########################################
uplo = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    icon="upload"
)
uplo.layout.min_width = '49%'
uplo.layout.min_height = '42px'
uplo.layout.border = 'thick double #CCCCFF'
uplo.observe(upload_func, names='value')
###############################

In [20]:
##################
Capture_w = widgets.ToggleButton(
    value=False,
    description='Capture',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour réaliser une capture Webcam",
    icon='camera' # (FontAwesome names without the `fa-` prefix)
)
Capture_w.observe(Capture_w_fonc,names='value')
########################################

In [21]:
# Contraste_Boite = HBox([ contraste_widg, clair_z,sombre_z], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
Contraste_Boite = HBox([ contraste_widg], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))

Cany_invert_Boite = HBox([Cany_w, Invert_w], layout=Layout(margin='0 0 0 0px'))
Controle_Boite = HBox([Stop_w, Rescue_w])
Img_src_Boite = HBox([uplo, Capture_w ])
Menu_Boite1 = VBox([Controle_Boite, Img_src_Boite, Contraste_Boite, Cany_invert_Boite])
Menus = HBox([Img, Menu_Boite1], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
display(Menus)

In [22]:
camera = CameraStream(constraints={'facingMode': {"exact": "environment"},'audio': False,'video': { 'width': 640, 'height': 480}})
# camera.layout.width='20%'
image_recorder = ImageRecorder(stream=camera, tooltip='Cliquer pour tenter une Capture')

btn_stop = Button(description="Envois à l'Iris",tooltip='Cette capture est Ok ?',button_style='success', icon="eye")
btn_stop.on_click(Recup_image)

yo=HBox([camera,VBox([btn_stop,image_recorder])], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))

yo.layout.display = 'none'
display(yo)

<div style="text-align:center; border:2px solid black; font-size: 1.8em; width:55%;">
<span style="color:blue"><b>Application Web de Travail sur l'encodage :</b></span>
</div>

In [23]:
def Dec_Bin_Hex(_):
    Nombre = Nombre_utilisateur.value
    if Nombre != "":
        try:
            Nombre = int(Nombre)
            if Nombre >=0:
                Pre_Nb_bin = bin(Nombre)
                Nb_bin = str(Pre_Nb_bin[2:])
                Pre_Nb_hex = hex(Nombre)
                Nb_hex = str(Pre_Nb_hex[2:])
                phrase_bin = "L'encodage, en binaire, est : " + Nb_bin
                Encod_Dec_Bin_label.value = phrase_bin
                phrase_hex = "L'encodage, en hexadécimal, est : " + Nb_hex
                Encod_Dec_Hex_label.value = phrase_hex
            else:
                phrase_bin = "L'encodage, en binaire, est : "
                Encod_Dec_Bin_label.value = phrase_bin 
                phrase_hex = "L'encodage, en hexadécimal, est : "
                Encod_Dec_Hex_label.value = phrase_hex
        except ValueError:
            phrase_bin = "ATTENTION, vous devez entrer un NOMBRE ENTIER POSITIF."
            Encod_Dec_Bin_label.value = phrase_bin
            phrase_hex = "ATTENTION, vous devez entrer un NOMBRE ENTIER POSITIF."
            Encod_Dec_Hex_label.value = phrase_hex
    else:
        phrase_bin = "L'encodage, en binaire, est : "
        Encod_Dec_Bin_label.value = phrase_bin 
        phrase_hex = "L'encodage, en hexadécimal, est : "
        Encod_Dec_Hex_label.value = phrase_hex

In [24]:
def Bin_Dec_Hex(_):
    Nombre = Nombre_utilisateur2.value
    if Nombre != "":
        try:
            Nombre = int(Nombre)
            Nombre_0b = "0b" + str(Nombre)
            if Nombre >=0:
                try:
                    Pre_Nb_dec = int(Nombre_0b,2)
                    Nb_dec = str(Pre_Nb_dec) 
                    Pre_Nb_hex = hex(Pre_Nb_dec)
                    Nb_hex = str(Pre_Nb_hex[2:])
                    phrase_dec = "L'encodage, en décimal, est : " + Nb_dec
                    Encod_Bin_Dec_label.value = phrase_dec
                    phrase_hex = "L'encodage, en hexadécimal, est : " + Nb_hex
                    Encod_Bin_Hex_label.value = phrase_hex
                except ValueError :
                    phrase_dec = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
                    Encod_Bin_Dec_label.value = phrase_dec
                    phrase_hex = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
                    Encod_Bin_Hex_label.value = phrase_hex
            else:
                phrase_dec = "L'encodage, en décimal, est : "
                Encod_Bin_Dec_label.value = phrase_dec
                phrase_hex = "L'encodage, en hexadécimal, est : " + Nb_hex
                Encod_Bin_Hex_label.value = phrase_hex
        except ValueError:
            phrase_dec = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
            Encod_Bin_Dec_label.value = phrase_dec
            phrase_hex = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
            Encod_Bin_Hex_label.value = phrase_hex
    else:
        phrase_dec = "L'encodage, en décimal, est : "
        Encod_Bin_Dec_label.value = phrase_dec 
        phrase_hex = "L'encodage, en hexadécimal, est : "
        Encod_Bin_Hex_label.value = phrase_hex

In [25]:
def Hex_Dec_Bin(_):
    Nombre = Nombre_utilisateur3.value
    if Nombre != "":
        try:
            Nombre = int(Nombre,16)
            if Nombre >=0:
                Pre_Nb_bin = bin(Nombre)
                Nb_bin = str(Pre_Nb_bin[2:])
                phrase_dec = "L'encodage, en décimal, est : " + str(Nombre)
                Encod_Hex_Dec_label.value = phrase_dec
                phrase_bin = "L'encodage, en binaire, est : " + Nb_bin
                Encod_Hex_Bin_label.value = phrase_bin
            else:
                phrase_dec = "ATTENTION, vous devez entrer un NOMBRE HEXADECIMAL ENTIER POSITIF."
                Encod_Hex_Dec_label.value = phrase_dec
                phrase_bin = "ATTENTION, vous devez entrer un NOMBRE HEXADECIMAL ENTIER POSITIF."
                Encod_Hex_bin_label.value = phrase_bin
        except ValueError :
            phrase_dec = "ATTENTION, vous devez entrer un NOMBRE HEXADECIMAL ENTIER POSITIF."
            Encod_Hex_Dec_label.value = phrase_dec
            phrase_bin = "ATTENTION, vous devez entrer un NOMBRE HEXADECIMAL ENTIER POSITIF."
            Encod_Hex_Bin_label.value = phrase_bin
    else:
        phrase_dec = "L'encodage, en décimal, est : "
        Encod_Hex_Dec_label.value = phrase_dec 
        phrase_bin = "L'encodage, en binaire, est : "
        Encod_Hex_Bin_label.value = phrase_bin

In [26]:
Nombre_utilisateur_label = widgets.Label(value="Entrez un nombre décimal entier positif :")
Text_multi_ligne1 = "L'encodage, en binaire, est : "
Encod_Dec_Bin_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne1 +' </p>')
Text_multi_ligne2 = "L'encodage, en hexadécimal, est : "
Encod_Dec_Hex_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne2 +' </p>')
Nombre_utilisateur = widgets.Textarea(
    disabled=False
)
Nombre_utilisateur.observe(Dec_Bin_Hex, names='value')
Conv_Dec_Bin = VBox([Nombre_utilisateur_label,Nombre_utilisateur , Encod_Dec_Bin_label, Encod_Dec_Hex_label  ])
Conv_Dec_Bin.layout.max_width = '33%'
Conv_Dec_Bin.layout.min_height = '150px'
Conv_Dec_Bin.layout.border = 'thick double #0066FF'

In [27]:
Nombre_utilisateur_label2 = widgets.Label(value="Entrez un nombre binaire (entier positif) :")
Text_multi_ligne1 = "L'encodage, en décimal, est : "
Encod_Bin_Dec_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne1 +' </p>')
Text_multi_ligne2 = "L'encodage, en hexadécimal, est : "
Encod_Bin_Hex_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne2 +' </p>')
Nombre_utilisateur2 = widgets.Textarea(
    disabled=False
)
Nombre_utilisateur2.observe(Bin_Dec_Hex, names='value')
Conv_Dec_Bin2 = VBox([Nombre_utilisateur_label2,Nombre_utilisateur2 , Encod_Bin_Dec_label, Encod_Bin_Hex_label  ])
Conv_Dec_Bin2.layout.max_width = '33%'
Conv_Dec_Bin2.layout.min_height = '150px'
Conv_Dec_Bin2.layout.border = 'thick double #33CC33'

In [28]:
Nombre_utilisateur_label3 = widgets.Label(value="Entrez un nombre hexadécimal (entier positif) :")
Text_multi_ligne1 = "L'encodage, en décimal, est : "
Encod_Hex_Dec_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne1 +' </p>')
Text_multi_ligne2 = "L'encodage, en binaire, est : "
Encod_Hex_Bin_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne2 +' </p>')
Nombre_utilisateur3 = widgets.Textarea(
    disabled=False
)
Nombre_utilisateur3.observe(Hex_Dec_Bin, names='value')
Conv_Dec_Bin3 = VBox([Nombre_utilisateur_label3, Nombre_utilisateur3 , Encod_Hex_Dec_label, Encod_Hex_Bin_label  ])
Conv_Dec_Bin3.layout.max_width = '33%'
Conv_Dec_Bin3.layout.min_height = '150px'
Conv_Dec_Bin3.layout.border = 'thick double #FF6633'

In [29]:
Menu = HBox([Conv_Dec_Bin,Conv_Dec_Bin2, Conv_Dec_Bin3])
display(Menu)

In [30]:
def Complet_a8(Nb):
    Longueur_Nb = len(Nb)
    if Longueur_Nb<8:
        Ajout = 8 - Longueur_Nb
        for i in range(0,Ajout):
            Nb = "0" + Nb
        Nb = "<span style='color:white'>.. </span>" + Nb
        return Nb
def Complet_a8_plus(Nb):
    Longueur_Nb = len(Nb)
    if Longueur_Nb<8:
        Ajout = 8 - Longueur_Nb
        for i in range(0,Ajout):
            Nb = "0" + Nb
        Nb = "+" + Nb
        # Somm_Visuel_label2 = widgets.HTML(value=f"<p style='font-size:28px'><b><u>{'+00000000'}</u></b></p>")
        return Nb
def Add_nb1(_):
    Nb1 = Nombre_utilisateur4.value
    if Nb1 =="":
        Somm_control_widg1.value = 0
        Somm_ok_widg.value = 0
        control_label.value = str(Somm_ok_widg.value)
        Nb1 = Complet_a8(Nb1)
        Somm_Visuel_label1.value = f"<p style='font-size:28px'><b>{Nb1}</b></p>"
        Nombre_utilisateur_label4.value="Entrez votre 1er nombre binaire :"
        
        vals = ['.','0','0','0','0','0','0','0','0']
        colors = ['white','white','white','white','white','white','white','white','white','white','white','white']
        Ref = f"<p style='font-size:24px'><b><span style='color:{colors[0]}'>{vals[0]}</span><span style='color:{colors[1]}'>{vals[1]}</span><span style='color:{colors[2]}'>{vals[2]}</span><span style='color:{colors[3]}'>{vals[3]}</span><span style='color:{colors[4]}'>{vals[4]}</span><span style='color:{colors[5]}'>{vals[5]}</span><span style='color:{colors[6]}'>{vals[6]}</span><span style='color:{colors[7]}'>{vals[7]}</span><span style='color:{colors[8]}'>{vals[8]}</span></b></p>"
        Retenue_label.value=Ref
        
        Somm_Visuel_label4.value=f"<b><span style='font-size:24px ; color:white'>{'...'}</span><span style='font-size:28px'>{'00000000'}</span></b>"

        
    else:
        try:
            debin_Nb1 = int(Nb1,2)
            Somm_control_widg1.value = 1
            Nb1 = Complet_a8(Nb1)
            Somm_Visuel_label1.value = f"<p style='font-size:28px'><b>{Nb1}</b></p>"
            if Somm_control_widg2.value == 1:
                Somm_ok_widg.value = 1
                control_label.value = str(Somm_ok_widg.value)

                
        except ValueError:
            Nombre_utilisateur_label4.value = "Attention, Vous devez entrer un NOMBRE BINAIRE : "
def Add_nb2(_):
    Nb2 = Nombre_utilisateur5.value
    if Nb2 =="":
        Somm_control_widg2.value = 0
        Somm_ok_widg.value = 0
        control_label.value = str(Somm_ok_widg.value)
        Nb2 = Complet_a8_plus(Nb2)
        Somm_Visuel_label2.value = f"<p style='font-size:28px'><b><u>{Nb2}</u></b></p>"
        Nombre_utilisateur_label5.value="Entrez votre 2nd nombre binaire :"
        
        vals = ['.','0','0','0','0','0','0','0','0']
        colors = ['white','white','white','white','white','white','white','white','white','white','white','white']
        Ref = f"<p style='font-size:24px'><b><span style='color:{colors[0]}'>{vals[0]}</span><span style='color:{colors[1]}'>{vals[1]}</span><span style='color:{colors[2]}'>{vals[2]}</span><span style='color:{colors[3]}'>{vals[3]}</span><span style='color:{colors[4]}'>{vals[4]}</span><span style='color:{colors[5]}'>{vals[5]}</span><span style='color:{colors[6]}'>{vals[6]}</span><span style='color:{colors[7]}'>{vals[7]}</span><span style='color:{colors[8]}'>{vals[8]}</span></b></p>"
        Retenue_label.value=Ref
        
        Somm_Visuel_label4.value=f"<b><span style='font-size:24px ; color:white'>{'...'}</span><span style='font-size:28px'>{'00000000'}</span></b>"
        
    else:
        try:
            debin_Nb2 = int(Nb2,2)
            Somm_control_widg2.value = 1
            Nb2 = Complet_a8_plus(Nb2)
            Somm_Visuel_label2.value = f"<p style='font-size:28px'><b><u>{Nb2}</u></b></p>"
            if Somm_control_widg1.value == 1:
                Somm_ok_widg.value = 1
                control_label.value = str(Somm_ok_widg.value)
        except ValueError:
            Nombre_utilisateur_label5.value = "Attention, Vous devez entrer un NOMBRE BINAIRE : "
def Recalcul_retenu():
    for i in range(0,6):
        j = 7 - i
        print("j=",j)
        Nb1 = Somm_Visuel_label1.value
        Nb1=Nb1[65:73] # 31:39
        print("Nb1=",Nb1)
        Nb1 = int(Nb1[j])
        print("Nb1=",Nb1)
        Nb2 = Somm_Visuel_label2.value
        Nb2=Nb2[33:41]
        print("Nb2=",Nb2)
        Nb2 = int(Nb2[j])
        print("Nb2=",Nb2)
        index = j
        b = vals[index+1]
        c = vals[index]
        print("yo nb2 =",Nb2)
        print("index = ",index," et vals[index+1]=",b, " et vals[index]=",c)
        if Nb1 == 1 and Nb2 == 1: # or (vals[index+1]==1 and Nb2==1) or (vals[index+1]==1 and Nb1==1):
            print("oui")
#             index = j
#             print("index=",index)
            vals[index] = "1"
            colors[index] = "red"
            print("vals=",vals)
#             vals[index-1] = "1"
#             colors[index-1] = "red"
#             print("vals=",vals)
        else:
            if (vals[index+1]=="1" and Nb2==1) :
                print("oui")
    #             index = j
    #             print("index=",index)
                vals[index] = "1"
                colors[index] = "red"
                print("vals=",vals)
    #             vals[index-1] = "1"
    #             colors[index-1] = "red"
    #             print("vals=",vals)
            else:
                if (vals[index+1]=="1" and Nb1==1):
                    print("oui")
        #             index = j
        #             print("index=",index)
                    vals[index] = "1"
                    colors[index] = "red"
                    print("vals=",vals)
        #             vals[index-1] = "1"
        #             colors[index-1] = "red"
        #             print("vals=",vals)    
    
                else:
                    index = j
                    print("non")
        #             print("index=",index)
                    vals[index] = "0" # vals[index-1] = "0"
                    colors[index] = "white"
    Ref = f"<p style='font-size:28px'><b><span style='color:white'>{'..'}</span><span style='color:{colors[0]}'>{vals[0]}</span><span style='color:{colors[1]}'>{vals[1]}</span><span style='color:{colors[2]}'>{vals[2]}</span><span style='color:{colors[3]}'>{vals[3]}</span><span style='color:{colors[4]}'>{vals[4]}</span><span style='color:{colors[5]}'>{vals[5]}</span><span style='color:{colors[6]}'>{vals[6]}</span><span style='color:{colors[7]}'>{vals[7]}</span><span style='color:{colors[8]}'>{vals[8]}</span></b></p>"
#     print("Ref=",Ref)
    Retenue_label.value=Ref
    return vals,colors
def Somm_Bin(_):
    Ok = Somm_ok_widg.value
    if Ok == 1:
        Nb1 = Nombre_utilisateur4.value
        Nb2 = Nombre_utilisateur5.value
        somme = int(Nb1,2) + int(Nb2,2)
        pre_somme_bin = bin(somme)
        somme_bin = pre_somme_bin[2:]
        Addition_label.value = "Voici la somme : " + somme_bin + ""
        Somm_ok_widg.value = 0
        val = Complet_a8(str(somme_bin))
        Somm_Visuel_label4.value = f"<p style='font-size:28px'><b>{val}</b></p>"
        Recalcul_retenu()


In [31]:
Nombre_utilisateur_label4 = widgets.Label(value="Entrez votre 1er nombre binaire :")
Nombre_utilisateur4 = widgets.Textarea(disabled=False)
Nombre_utilisateur_label5 = widgets.Label(value="Entrez votre 2nd nombre binaire :")
Nombre_utilisateur5 = widgets.Textarea(disabled=False)

Text_multi_ligne1 = "La somme de ces 2 nombres binaire est : "
Addition_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne1 +' </p>')
control_label = widgets.Label(value="")

Nombre_utilisateur4.observe(Add_nb1, names='value')
Nombre_utilisateur5.observe(Add_nb2, names='value')

Somm_control_widg1 = widgets.IntSlider(min = 0, max = 1,  value = 0 )
# Somm_control_widg1.observe(Test_Somm_control_widg1, names='value')
Somm_control_widg2 = widgets.IntSlider(min = 0, max = 1,  value = 0 )
# Somm_control_widg2.observe(Test_Somm_control_widg2, names='value')

Somm_ok_widg = widgets.IntSlider(min = 0, max = 1,  value = 0 )
Somm_ok_widg.observe(Somm_Bin, names='value')

# Retenues ##
# Somm_Visuel_label0 = widgets.HTML(value=f"<p style='color:white'>{'_____________'}</p>")
vals = ['.','0','0','0','0','0','0','0','0']
colors = ['white','white','white','white','white','white','white','white','white','white','white','white']
Ref = f"<p style='font-size:24px'><b><span style='color:{colors[0]}'>{vals[0]}</span><span style='color:{colors[1]}'>{vals[1]}</span><span style='color:{colors[2]}'>{vals[2]}</span><span style='color:{colors[3]}'>{vals[3]}</span><span style='color:{colors[4]}'>{vals[4]}</span><span style='color:{colors[5]}'>{vals[5]}</span><span style='color:{colors[6]}'>{vals[6]}</span><span style='color:{colors[7]}'>{vals[7]}</span><span style='color:{colors[8]}'>{vals[8]}</span></b></p>"
Retenue_label = widgets.HTML(value=Ref)
###

Somm_Visuel_label1 = widgets.HTML(value = f"<b><span style='font-size:24px ; color:white'>{'...'}</span><span style='font-size:28px'>{'00000000'}</span></b>")

Somm_Visuel_label2 = widgets.HTML(value=f"<p style='font-size:28px'><b><span style='color:white'>{''}</span><u>{'+00000000'}</u></b></p>")
Somm_Visuel_label4 = widgets.HTML(value=f"<b><span style='font-size:24px ; color:white'>{'...'}</span><span style='font-size:28px'>{'00000000'}</span></b>")
Somm_Visuel_label4.border = 'thick double #000000'
Box_Somm_Visuel = VBox([Retenue_label,Somm_Visuel_label1, Somm_Visuel_label2, Somm_Visuel_label4])
Box_Somm_Interact = VBox([Nombre_utilisateur_label4, Nombre_utilisateur4, Nombre_utilisateur_label5, Nombre_utilisateur5,Addition_label,control_label])
Box_espace = widgets.HTML(value=f"<p style='color:white'>{'_____________'}</p>")
# Box_espace.width = '150px'
Somm_Titre = widgets.HTML(value=f"<p style='font-size:20px; color:purple;'><b><u>{'Addition de 2 nombres binaires sur 7 bits : '}</u></b></p>")
Somm = VBox([Somm_Titre,HBox([ Box_Somm_Interact,Box_espace, Box_Somm_Visuel])])
Somm.layout.max_width = '65%'
Somm.layout.min_height = '150px'
Somm.layout.border = 'thick double #000000'

In [32]:
display(Somm)

j= 7
Nb1= 00000001
Nb1= 1
Nb2= 00000001
Nb2= 1
yo nb2 = 1
index =  7  et vals[index+1]= 0  et vals[index]= 0
oui
vals= ['.', '0', '0', '0', '0', '0', '0', '1', '0']
j= 6
Nb1= 00000001
Nb1= 0
Nb2= 00000001
Nb2= 0
yo nb2 = 0
index =  6  et vals[index+1]= 1  et vals[index]= 0
non
j= 5
Nb1= 00000001
Nb1= 0
Nb2= 00000001
Nb2= 0
yo nb2 = 0
index =  5  et vals[index+1]= 0  et vals[index]= 0
non
j= 4
Nb1= 00000001
Nb1= 0
Nb2= 00000001
Nb2= 0
yo nb2 = 0
index =  4  et vals[index+1]= 0  et vals[index]= 0
non
j= 3
Nb1= 00000001
Nb1= 0
Nb2= 00000001
Nb2= 0
yo nb2 = 0
index =  3  et vals[index+1]= 0  et vals[index]= 0
non
j= 2
Nb1= 00000001
Nb1= 0
Nb2= 00000001
Nb2= 0
yo nb2 = 0
index =  2  et vals[index+1]= 0  et vals[index]= 0
non
j= 7
Nb1= 00000010
Nb1= 0
Nb2= 00000001
Nb2= 1
yo nb2 = 1
index =  7  et vals[index+1]= 0  et vals[index]= 1
non
j= 6
Nb1= 00000010
Nb1= 1
Nb2= 00000001
Nb2= 0
yo nb2 = 0
index =  6  et vals[index+1]= 1  et vals[index]= 0
oui
vals= ['.', '0', '0', '0', '0', '0', '1', '1